In [4]:
import pandas as pd
import json
#skip warning
import warnings
warnings.filterwarnings('ignore')
#connect to drive
from google.colab import drive  
import pandas as pd
import numpy as np
import json
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
def CustomParser(data):
    j1 = json.loads(data.replace("'",'"'))
    return j1

In [65]:
tahun = '2017'
WordCount = pd.read_csv("/content/drive/MyDrive/Skripsi_Putri/Hasil/Join_CountWord_%s.csv"%tahun,
                        converters={'Innovation':CustomParser,
                                    'Integrity':CustomParser,
                                    'Quality':CustomParser,
                                    'Respect':CustomParser,
                                    'Teamwork':CustomParser})
WordCount.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation,Integrity,Quality,Respect,Teamwork
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,"{'ability': 5, 'agricultural technology': 1, '...","{'accountability': 9, 'accountability responsi...","{'best service': 1, 'capable': 1, 'characteris...","{'adviser': 1, 'advisor': 1, 'ceo': 2, 'commit...","{'align': 1, 'assist': 4, 'assisting': 3, 'ass..."
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,"{'ability': 9, 'allow': 4, 'best': 27, 'brand'...","{'accountability': 10, 'accountability respons...","{'build': 3, 'capable': 1, 'client': 3, 'clien...","{'best young': 1, 'committee member': 2, 'comp...","{'align': 1, 'assist': 1, 'association': 19, '..."
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,"{'ability': 11, 'agility': 2, 'allow': 2, 'bes...","{'accountability': 6, 'accountability responsi...","{'ability service': 2, 'assistance service': 2...","{'advisor': 2, 'audit team': 6, 'best best': 1...","{'assist': 5, 'assisting': 9, 'association': 3..."
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,"{'ability': 25, 'accounting model': 1, 'achiev...","{'accountability': 20, 'accountability respons...","{'best quality': 2, 'best service': 1, 'build'...","{'accountability team': 1, 'advisor': 4, 'best...","{'align': 3, 'assist': 11, 'assisting': 5, 'as..."
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,"{'ability': 9, 'achievements operational': 1, ...","{'accountability': 4, 'accountability responsi...","{'base sales': 1, 'best product': 1, 'build': ...","{'advisor': 10, 'advisors': 1, 'audit team': 4...","{'assist': 4, 'association': 13, 'business par..."


In [93]:
Report = pd.read_csv("/content/drive/MyDrive/Skripsi_Putri/Hasil/Join_Report_%s.csv"%tahun)
Report.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation,Integrity,Quality,Respect,Teamwork
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,289,874,176,927,183
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,210,653,132,548,136
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,329,1735,528,1569,315
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,816,5027,963,3597,793
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,173,615,277,670,91


In [67]:
word = pd.read_csv('/content/drive/MyDrive/Skripsi_Putri/Hasil/Unique_Word_Per_Cultures_%s.csv'%tahun)
word.head()

,Innovation,Integrity,Quality,Respect,Teamwork
0,ability,able accountable,ability service,ability leadership,able put
1,abilty,accountability,acceptance customer,ability set,accountant business
2,acceleration innovation,accountability corporate,access customer,ability skill,accountant partner
3,acceleration technology,accountability governance,access provider,academy leadership,acquisition partnership
4,accountability operational,accountability responsibility,access service,accountability team,add together


In [68]:
word2 = []
word2 +=list(word['Innovation'].dropna())
word2 += list(word['Integrity'].dropna())
word2 += list(word['Quality'].dropna())
word2 += list(word['Respect'].dropna())
word2 += list(word['Teamwork'].dropna())
len(word2)

7711

In [69]:
df_res = pd.DataFrame(word2,columns=['word'])
df_res['df'] = 0
df_res.head()

,word,df
0,ability,0
1,abilty,0
2,acceleration innovation,0
3,acceleration technology,0
4,accountability operational,0


In [70]:
for i in range(len(df_res)) :
  for j in range(len(WordCount)) :
    cek = False
    for k in WordCount.columns[4:] :
      if df_res.iloc[i,0] in WordCount.loc[j,k].keys() :
        cek = True
        break
    if cek :
      df_res.iloc[i,1] += 1

In [71]:
df_res.head()

,word,df
0,ability,574
1,abilty,3
2,acceleration innovation,1
3,acceleration technology,1
4,accountability operational,5


In [72]:
import math
def idf(x) :
  return math.log(len(WordCount)/x)
df_res['idf'] = df_res['df'].apply(idf)
df_res.head()

,word,df,idf
0,ability,574,0.032565
1,abilty,3,5.286582
2,acceleration innovation,1,6.385194
3,acceleration technology,1,6.385194
4,accountability operational,5,4.775756


In [73]:
df_res.to_csv("/content/drive/MyDrive/Skripsi_Putri/Hasil/IDF_%s.csv"%tahun,index=False)

In [94]:
Report['Count All Word'] = 0

In [95]:
for i in range(len(Report)) :
  sum_word = 0
  for j in Report.columns[4:] :
    sum_word += Report.loc[i,j]
  Report.iloc[i,-1] = sum_word

In [96]:
Report.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation,Integrity,Quality,Respect,Teamwork,Count All Word
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,289,874,176,927,183,2449
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,210,653,132,548,136,1679
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,329,1735,528,1569,315,4476
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,816,5027,963,3597,793,11196
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,173,615,277,670,91,1826


In [97]:
Report.to_csv("CountWord_2018.csv",index=False)

In [77]:
WordCount2 = WordCount.copy()
for i in range(len(WordCount2)) :
  for j in WordCount2.columns[4:] :
    for key in WordCount2.loc[i,j].keys() :
      WordCount2.loc[i,j][key] /= Report.iloc[i,-1]

In [78]:
WordCount2.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation,Integrity,Quality,Respect,Teamwork
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,"{'ability': 0.001699524133242692, 'agricultura...","{'accountability': 0.0030591434398368456, 'acc...","{'best service': 0.0003399048266485384, 'capab...","{'adviser': 0.0003399048266485384, 'advisor': ...","{'align': 0.0003399048266485384, 'assist': 0.0..."
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,"{'ability': 0.004909983633387889, 'allow': 0.0...","{'accountability': 0.005455537370430988, 'acco...","{'build': 0.0016366612111292963, 'capable': 0....","{'best young': 0.0005455537370430987, 'committ...","{'align': 0.0005455537370430987, 'assist': 0.0..."
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,"{'ability': 0.0025575447570332483, 'agility': ...","{'accountability': 0.0013950244129272262, 'acc...","{'ability service': 0.00046500813764240875, 'a...","{'advisor': 0.00046500813764240875, 'audit tea...","{'assist': 0.0011625203441060219, 'assisting':..."
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,"{'ability': 0.002464025231618372, 'accounting ...","{'accountability': 0.001971220185294697, 'acco...","{'best quality': 0.00019712201852946975, 'best...","{'accountability team': 9.856100926473487e-05,...","{'align': 0.00029568302779420464, 'assist': 0...."
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,"{'ability': 0.005338078291814947, 'achievement...","{'accountability': 0.002372479240806643, 'acco...","{'base sales': 0.0005931198102016608, 'best pr...","{'advisor': 0.005931198102016607, 'advisors': ...","{'assist': 0.002372479240806643, 'association'..."


In [79]:
df_res.set_index('word',inplace=True)

In [80]:
for i in range(len(WordCount2)) :
  for j in WordCount2.columns[4:] :
    for key in WordCount2.loc[i,j].keys() :
      WordCount2.loc[i,j][key] *= df_res.loc[key,'idf']

In [81]:
WordCount2.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation,Integrity,Quality,Respect,Teamwork
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,"{'ability': 5.534500795064428e-05, 'agricultur...","{'accountability': 0.0002247234223668587, 'acc...","{'best service': 0.0004014914044598675, 'capab...","{'adviser': 0.0008475098915319846, 'advisor': ...","{'align': 0.0004167705981588023, 'assist': 0.0..."
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,"{'ability': 0.00015989363017099393, 'allow': 0...","{'accountability': 0.0004007615376148987, 'acc...","{'build': 0.0002923379251600621, 'capable': 0....","{'best young': 0.0031053176314445063, 'committ...","{'align': 0.0006689247680213837, 'assist': 8.2..."
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,"{'ability': 8.328645186229948e-05, 'agility': ...","{'accountability': 0.00010247792119713218, 'ac...","{'ability service': 0.0015534396471863763, 'as...","{'advisor': 0.00027254208069620993, 'audit tea...","{'assist': 0.00017528908774628468, 'assisting'..."
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,"{'ability': 8.024098826670387e-05, 'accounting...","{'accountability': 0.00014480502630556064, 'ac...","{'best quality': 0.0003469784320349802, 'best ...","{'accountability team': 0.0005610139186317544,...","{'align': 0.0003625485215207559, 'assist': 0.0..."
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,"{'ability': 0.00017383453386917667, 'achieveme...","{'accountability': 0.0001742813519449844, 'acc...","{'base sales': 0.0033760659658587074, 'best pr...","{'advisor': 0.00347628555478766, 'advisors': 0...","{'assist': 0.00035773113470783887, 'associatio..."


In [82]:
WordCount2.to_csv('/content/drive/MyDrive/Skripsi_Putri/Hasil/TF_IDF_%s.csv'%tahun,index=False)

In [83]:
WordCount3 = WordCount2.copy()
WordCount3['Innovation_tf_idf_sum'] = 0
WordCount3['Integrity_tf_idf_sum'] = 0
WordCount3['Quality_tf_idf_sum'] = 0
WordCount3['Respect_tf_idf_sum'] = 0
WordCount3['Teamwork_tf_idf_sum'] = 0
for i in range(len(WordCount3)) :
  for j in WordCount3.columns[4:9] :
    WordCount3.loc[i,j+'_tf_idf_sum'] = sum(WordCount3.loc[i,j].values())

In [84]:
WordCount3.drop(['Innovation','Integrity','Respect','Quality','Teamwork'],axis=1,inplace=True)
WordCount3.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation_tf_idf_sum,Integrity_tf_idf_sum,Quality_tf_idf_sum,Respect_tf_idf_sum,Teamwork_tf_idf_sum
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,0.046897,0.035330,0.038560,0.034965,0.036134
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,0.066069,0.053999,0.046734,0.051809,0.030025
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,0.044543,0.062816,0.087488,0.029986,0.021018
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,0.106214,0.048028,0.058157,0.059744,0.020820
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,0.071963,0.023302,0.095000,0.079762,0.017967


In [85]:
WordCount3.to_csv('/content/drive/MyDrive/Skripsi_Putri/Hasil/Rank_Document_sum_%s.csv'%tahun,index=False)

In [86]:
WordCount4 = WordCount2.copy()
WordCount4['Innovation_tf_idf_mean'] = 0
WordCount4['Integrity_tf_idf_mean'] = 0
WordCount4['Quality_tf_idf_mean'] = 0
WordCount4['Respect_tf_idf_mean'] = 0
WordCount4['Teamwork_tf_idf_mean'] = 0
for i in range(len(WordCount4)) :
  for j in WordCount4.columns[4:9] :
    WordCount4.loc[i,j+'_tf_idf_mean'] = WordCount3.loc[i,j+'_tf_idf_sum']/len(WordCount4.loc[i,j].keys())

In [87]:
WordCount4.drop(['Innovation','Integrity','Respect','Quality','Teamwork'],axis=1,inplace=True)
WordCount4.head()

,Code,Company Name,Sektor,Sub Sektor,Innovation_tf_idf_mean,Integrity_tf_idf_mean,Quality_tf_idf_mean,Respect_tf_idf_mean,Teamwork_tf_idf_mean
0,AALI,Astra Agro Lestari Tbk.,IDXNONCYC,Plantations & Crops,0.000809,0.000520,0.000676,0.000648,0.000821
1,ABBA,Mahaka Media Tbk.,IDXCYCLIC,Consumer Publishing,0.001271,0.000844,0.001062,0.001205,0.000790
2,ABDA,Asuransi Bina Dana Arta Tbk.,IDXFINANCE,General Insurance,0.000730,0.000668,0.000767,0.000555,0.000447
3,ABMM,ABM Investama Tbk.,IDXINDUST,Multi-sector Holdings,0.001398,0.000404,0.000410,0.000586,0.000330
4,ACES,Ace Hardware Indonesia Tbk.,IDXCYCLIC,Home Improvement Retail,0.001564,0.000496,0.001439,0.001564,0.000781


In [88]:
WordCount4.to_csv('/content/drive/MyDrive/Skripsi_Putri/Hasil/Rank_Document_average_%s.csv'%tahun,index=False)